# 5 - Multi-class Sentiment Analysis

In all of the previous notebooks we have performed sentiment analysis on a dataset with only two classes, positive or negative. When we have only two classes our output can be a single scalar, bound between 0 and 1, that indicates what class an example belongs to. When we have more than 2 examples, our output must be a $C$ dimensional vector, where $C$ is the number of classes.

In this notebook, we'll be performing classification on a dataset with 6 classes. Note that this dataset isn't actually a sentiment analysis dataset, it's a dataset of questions and the task is to classify what category the question belongs to. However, everything covered in this notebook applies to any dataset with examples that contain an input sequence belonging to one of $C$ classes.

Below, we setup the fields, and load the dataset. 

The first difference is that we do not need to set the `dtype` in the `LABEL` field. When doing a mutli-class problem, PyTorch expects the labels to be numericalized `LongTensor`s. 

The second different is that we use `TREC` instead of `IMDB` to load the `TREC` dataset. The `fine_grained` argument allows us to use the fine-grained labels (of which there are 50 classes) or not (in which case they'll be 6 classes). You can change this how you please.

In [1]:
import torch
from torchtext import data, vocab
from torchtext import datasets
import random
import numpy as np

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# TEXT = data.Field(tokenize = 'spacy')
# LABEL = data.LabelField()

# train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

def tokenizer(x):
    return x.split()


TEXT = data.Field(tokenize=tokenizer, use_vocab=True, unk_token='<unk>', batch_first=True)
TERM = data.Field(tokenize=tokenizer, use_vocab=True, unk_token='<unk>', batch_first=True)
ASPECT = data.Field(unk_token=None, batch_first=True)
LABEL = data.Field(unk_token=None, batch_first=True)

fields = (('LABEL', LABEL), ('ASPECT', ASPECT), ('TERM', TERM), ('TEXT', TEXT))
# if train_type == 3:
#     fields = (('LABEL', LABEL), ('ASPECT', ASPECT), ('TERM', TERM), ('TEXT', TEXT))

train_data, test_data = data.TabularDataset.splits(path='../data/nepcls/kfold/ss_ac_at_txt_bal_lstm_3/1/', 
                                            format='csv', 
                                            train='train.txt',
                                            test = 'test.txt',
                                            fields=fields)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))


In [2]:
vars(train_data[0])

{'LABEL': ['1'],
 'ASPECT': ['GENERAL'],
 'TERM': ['हनुमान'],
 'TEXT': ['यो', 'बिस्नु', 'पौडेल', 'को', 'हनुमान', 'हो', '।']}

Let's look at one of the examples in the training set.

Next, we'll build the vocabulary. As this dataset is small (only ~3800 training examples) it also has a very small vocabulary (~7500 unique tokens), this means we do not need to set a `max_size` on the vocabulary as before.

In [3]:
vec = vocab.Vectors(name='nep_english.vec', cache='../data/embeddings')

TEXT.build_vocab(train_data, test_data, max_size=None, vectors=vec)
TERM.build_vocab(train_data, test_data, max_size=None, vectors=vec)
ASPECT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [4]:
print(ASPECT.vocab.itos)

['<pad>', 'FEEDBACK', 'GENERAL', 'PROFANITY', 'VIOLENCE']


In [5]:
print(LABEL.vocab.itos)

['<pad>', '0', '1']


As always, we set up the iterators.

In [6]:
BATCH_SIZE = 2

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda:0')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.TEXT), 
    sort_within_batch=True,    
    device = device)

In [7]:
vars(train_iterator.dataset.examples[0])

{'LABEL': ['1'],
 'ASPECT': ['GENERAL'],
 'TERM': ['हनुमान'],
 'TEXT': ['यो', 'बिस्नु', 'पौडेल', 'को', 'हनुमान', 'हो', '।']}

We'll be using the CNN model from the previous notebook, however any of the models covered in these tutorials will work on this dataset. The only difference is now the `output_dim` will be $C$ instead of $1$.

In [8]:
# Check hidden shape
# Check pytorch LSTM (input, output shape)
#

import torch.nn as nn
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.ac_embeddings = nn.Embedding(5, embedding_dim)        
        
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, aspect, ac):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
        aspect = aspect.permute(1, 0)
        ac = ac.permute(1, 0)
                
        #text = [batch size, sent len]
#         print("Text shape ", text.type())
#         print("Aspect shape ", aspect.type())
#         print("AC type ", ac.type())
        
        embedded = self.embedding(text)
        aspect_emb = self.embedding(aspect)
        ac_emb = self.ac_embeddings(ac)
#         print("Shape of embedded", embedded.shape)
#         print("Shape of aspect_emb", aspect_emb.shape)
        
        embedded = torch.cat((embedded, aspect_emb), dim=0)
        embedded = torch.cat((embedded, ac_emb), dim=0)
        
#         print("Shape of embedded", embedded.shape)
                
        #embedded = [batch size, sent len, emb dim]
        
#         embedded = embedded.unsqueeze(1)
#         print("Shape of embedded after squeeze", embedded.shape)
        
        embedded, (hidden, cell) = self.lstm(self.dropout(embedded))
        
#         embedded = [batch size, sent_len, num_dim * hidden_dim]
#         hidden = [num_dim, sent_len, hidden_dim]

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
#         print("hidden shape ", hidden.shape)
                
        #hidden = [batch size, hid dim * num directions]
        final = self.fc(hidden)
#         print("final shape ", final)    
        
        return final
    
    
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
#         print("Shape of text", text.shape)
        
#         text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
#         print("Shape of embedded", embedded.shape)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
#         print("Shape of embedded after squeeze", embedded.shape)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
#         print("Shape of conved", conved[0].shape)
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
#         print("Shape of pooled", pooled[0].shape)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
#         print("Shape of cat", cat.shape)

        #cat = [batch size, n_filters * len(filter_sizes)]
        final = self.fc(cat)
#         print("Shape of final", final.shape)
        
        return final    

We define our model, making sure to set `OUTPUT_DIM` to $C$. We can get $C$ easily by using the size of the `LABEL` vocab, much like we used the length of the `TEXT` vocab to get the size of the vocabulary of the input.

The examples in this dataset are generally a lot smaller than those in the IMDb dataset, so we'll use smaller filter sizes.

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL=True
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

# N_FILTERS = 100
# FILTER_SIZES = [2,3,4]

# model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

/home/osingh1/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Checking the number of parameters, we can see how the smaller filter sizes means we have about a third of the parameters than we did for the CNN model on the IMDb dataset.

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,593,923 trainable parameters


Next, we'll load our pre-trained embeddings.

In [11]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0464, -0.2257,  0.0505,  ..., -0.0736,  0.0517, -0.0229],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

Then zero the initial weights of the unknown and padding tokens.

In [12]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

Another different to the previous notebooks is our loss function (aka criterion). Before we used `BCEWithLogitsLoss`, however now we use `CrossEntropyLoss`. Without going into too much detail, `CrossEntropyLoss` performs a *softmax* function over our model outputs and the loss is given by the *cross entropy* between that and the label.

Generally:
- `CrossEntropyLoss` is used when our examples exclusively belong to one of $C$ classes
- `BCEWithLogitsLoss` is used when our examples exclusively belong to only 2 classes (0 and 1) and is also used in the case where our examples belong to between 0 and $C$ classes (aka multilabel classification).

In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

Before, we had a function that calculated accuracy in the binary label case, where we said if the value was over 0.5 then we would assume it is positive. In the case where we have more than 2 classes, our model outputs a $C$ dimensional vector, where the value of each element is the beleief that the example belongs to that class. 

For example, in our labels we have: 'HUM' = 0, 'ENTY' = 1, 'DESC' = 2, 'NUM' = 3, 'LOC' = 4 and 'ABBR' = 5. If the output of our model was something like: **[5.1, 0.3, 0.1, 2.1, 0.2, 0.6]** this means that the model strongly believes the example belongs to class 0, a question about a human, and slightly believes the example belongs to class 3, a numerical question.

We calculate the accuracy by performing an `argmax` to get the index of the maximum value in the prediction for each element in the batch, and then counting how many times this equals the actual label. We then average this across the batch.

In [14]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True).squeeze(1) # get the index of the max probability
    correct = max_preds.eq(y)
    return correct.sum().item() / torch.FloatTensor([y.shape[0]])


The training loop is similar to before, without the need to `squeeze` the model predictions as `CrossEntropyLoss` expects the input to be **[batch size, n classes]** and the label to be **[batch size]**.

The label needs to be a `LongTensor`, which it is by default as we did not set the `dtype` to a `FloatTensor` as before.

In [15]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.TEXT, batch.TERM, batch.ASPECT)
        
        gold = batch.LABEL
        
        gold = gold.squeeze(1)
        
        loss = criterion(predictions, gold)
        
        acc = categorical_accuracy(predictions, gold)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The evaluation loop is, again, similar to before.

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.TEXT, batch.TERM, batch.ASPECT)
            
            gold = batch.LABEL
            
            gold = gold.squeeze(1)            
            
            loss = criterion(predictions, gold)
            
            acc = categorical_accuracy(predictions, gold)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Next, we train our model.

In [18]:
N_EPOCHS = 60

best_valid_loss = float('inf')
counter=0

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
        print("Best model saved!!!")
        counter=0
    else:
        counter += 1
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    if counter >= 5: 
        print("Training stopped because maximum tolerance reached!!!")
        break

Best model saved!!!
Epoch: 01 | Epoch Time: 0m 3s
	Train Loss: 0.623 | Train Acc: 68.97%
	 Val. Loss: 0.591 |  Val. Acc: 70.49%
Best model saved!!!
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 0.580 | Train Acc: 72.01%
	 Val. Loss: 0.524 |  Val. Acc: 75.37%
Best model saved!!!
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 0.399 | Train Acc: 81.97%
	 Val. Loss: 0.498 |  Val. Acc: 76.10%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 0.187 | Train Acc: 93.29%
	 Val. Loss: 0.604 |  Val. Acc: 74.15%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 0.095 | Train Acc: 96.44%
	 Val. Loss: 0.687 |  Val. Acc: 75.12%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 0.064 | Train Acc: 97.69%
	 Val. Loss: 0.711 |  Val. Acc: 77.07%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 0.028 | Train Acc: 98.85%
	 Val. Loss: 0.830 |  Val. Acc: 76.83%
Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 0.017 | Train Acc: 99.37%
	 Val. Loss: 0.897 |  Val. Acc: 73.17%
Training stopped because maximum tolerance reached!!!


Finally, let's run our model on the test set!

In [19]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.557 | Test Acc: 75.88%


Similar to how we made a function to predict sentiment for any given sentences, we can now make a function that will predict the class of question given.

The only difference here is that instead of using a sigmoid function to squash the input between 0 and 1, we use the `argmax` to get the highest predicted class index. We then use this index with the label vocab to get the human readable label.

In [ ]:
def tokenizer(x):
    return x.split()

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok for tok in tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    tensor = tensor.permute(1, 0)
    preds = model(tensor, None, None)
    max_preds = preds.argmax(dim = 1)
    
    return max_preds.item()

pred_class = predict_class(model, "राताे काेट लगाउने सर ले राम्रो कुरा गरनु भयाे निलाे काेट ले गाेबर कुरा गरेयाे निलाे काेटे छि छि छि ।")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

In [ ]:
pred_class = predict_class(model, "यो साले लाई मार्नु पर्छ ।")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')